# Setup

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Drivers in Google Colab
!pip install selenium
!apt-get update 
!pip install webdriver-manager
!apt install chromium-chromedriver

     |████████████████████████████████| 904 kB 5.3 MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:8 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [581 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,428 kB]

In [4]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException,ElementNotInteractableException
from selenium import webdriver
import time
import pandas as pd
import time
import re
import unicodedata
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

In [5]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver =webdriver.Chrome('chromedriver',options=chrome_options)
#wait = WebDriverWait(driver, 20)

In [12]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException,ElementNotInteractableException
from selenium import webdriver
import time
import pandas as pd
import time

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
path  = '/usr/lib/chromium-browser/chromedriver'

titles = pd.read_excel('/content/drive/MyDrive/Semantic Titles.xlsx',sheet_name='Sheet1')
certificates = pd.read_csv('/content/drive/MyDrive/data/certificates.csv') 

# Scrapping

In [13]:
def get_jobs(keyword, num_jobs, verbose,path,slp_time):
    
    """
    Get jobs from glassdoor, by passing a keyword. The returned dataframe includes
    -Job Title -> Title of the job, as given by employer
    -Searched Job Title -> The keyword passed as argument, the searched element
    -Salary Estimate -> The salary estimates given
    -Job Function -> The role of the job as given by employer
    -Job Description -> Description of job which comes before clicking 'See More'
    -Full Description -> Full Description that comes after clicking 'See More'
    -Company Name -> Name of the Company
    -Rating -> Rating of the company
    -Location -> Location of the job, as passed by user
    -Size -> Number of employees in the company
    -Founded -> When the company was founded
    -Type of ownership -> Kind of company
    -Industry -> Type of Industry
    -Sector -> Sector of the company
    -Revenue -> Revenue of the company
    -URL -> Link of individual jobs
    
    All the above data can be commented out as needed.
    
    
    Params: 
        - keyword = Job title to be searched
        - num_jobs = Number of jobs to be collected
        - verbose = Can be used to print the data as it is collected
        - path = path of the chromedriver
        - slp_time = time the browser needs to stop, depends on the network connection
    
    Returns:
        - Dataframe containing all the information
    """
    
    startTime = time.time()
    #Initializing the webdriver
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
    
    
    
    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=N&locId=115&locKeyword=India&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=false&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []
    pg=0
    
    print("Extracting: "+keyword)

    while len(jobs) < num_jobs:
        executionTime = (time.time() - startTime)
        if executionTime>250:
            break


        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(3)

        pg = pg+1

        try:
            time.sleep(5)
            driver.find_element_by_class_name("ModalStyle__xBtn___29PT9").click()
            print("worked")
              #clicking to the X.
    
        except NoSuchElementException:
            pass
        
        # found_popup = False 
        currentJoblist = 0
        #pg = 0
        num = 1
        
        if not (len(jobs) >= num_jobs) and (pg<5):
            listButtonsCount = len(driver.find_elements_by_xpath('//*[@id="MainCol"]//div[1]//ul//li[@data-test="jobListing"]'))
            
            print("No of job butons:" +str(listButtonsCount))
            #pg = pg+1

            #Going through each job in this page
            job_buttons = driver.find_elements_by_xpath('.//*[@id="MainCol"]//a[@class="jobLink"]')  #jl for Job Listing. These are the buttons we're going to click.
            
            for job_button in job_buttons:  
                
                key = True
    
                #print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
                if len(jobs) >= num_jobs:
                    break
                
                try:            
                    job_button.click()  #You might 
                except (StaleElementReferenceException,ElementNotInteractableException) as e:
                    driver.refresh()
                
                time.sleep(4)
                
                #code to kill the sign-up pop-up after it render on screen
                # if not found_popup:
                try:
                    driver.find_element_by_css_selector('[alt="Close"]').click()
                    print("worked")
                    # print("&&& line 89")
                    # found_popup = True
                except (NoSuchElementException,StaleElementReferenceException) as e:
                    # print("&&& line 92")
                    pass
                          
                
                collected_successfully = False
                
                while not collected_successfully:
                    try:
                        # company_name
                        try:
                            company_name = driver.find_element_by_xpath('//*[@id="MainCol"]//li['+ str(currentJoblist + 1) +']//div[2]//a//span').text
                        except (NoSuchElementException,StaleElementReferenceException) as e:
                            company_name = -1
                        
                        # location
                        try:
                            location = driver.find_element_by_xpath('//*[@id="MainCol"]//li['+ str(currentJoblist + 1) +']//div[2]//div[2]/span').text
                        except (NoSuchElementException,StaleElementReferenceException) as e:
                            location = -1
                        
                        # job_title
                        try:
                            job_title = driver.find_element_by_xpath('//*[@id="MainCol"]//li['+ str(currentJoblist + 1) +']//a[@data-test="job-link"]').text
                        except (NoSuchElementException,StaleElementReferenceException) as e:
                            job_title=-1
                            
                            
                        if job_title != keyword:
                            key = False
                                
                        #parent
                        try:
                            job_description = driver.find_elements_by_xpath('.//div[@class="jobDescriptionContent desc"]')
                        except (NoSuchElementException,StaleElementReferenceException) as e:
                            job_description = -1
                        
                        #getting child
                        if job_description!=-1:
                            t = ' '
                            for element in job_description:
                                t = t + element.text
                            full_description = t
                        
                        #job function
                        try:
                            job_function = driver.find_element_by_xpath('//*[@id="JDCol"]//strong[text()[1]="Job Function :"]//following-sibling::*').text
                        except (NoSuchElementException,StaleElementReferenceException) as e:
                            job_function = -1
                        
                        #try:
                            #job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                        #except (NoSuchElementException,StaleElementReferenceException) as e:
                            #job_description = -1
                            
                        
                        collected_successfully = True
                    except:
                        collected_successfully=False
                        #time.sleep(5)
    
                try:
                    #salary_estimate
                    salary_estimate = driver.find_element_by_xpath('//*[@id="JDCol"]//span[@data-test="detailSalary"]').text
                except (NoSuchElementException,StaleElementReferenceException) as e:
                    salary_estimate = -1 #You need to set a "not found value. It's important."
                    
                try:
                    link = driver.find_element_by_xpath('//span[contains(@class, "indeed-apply-widget indeed-apply-button-container indeed-apply-status-not-applied")]').get_attribute('data-indeed-apply-joburl')
                #link = driver.find_element_by_xpath('//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[2]/div/div[1]/div[1]/span/indeed-apply-widget indeed-apply-button-container indeed-apply-status-not-applied').text
                except (NoSuchElementException,StaleElementReferenceException) as e:
                    link = '-1'

        
                
                #try:
                    # rating
                 #   rating = driver.find_element_by_xpath('//*[@id="JDCol"]//span[@data-test="detailRating"]').text
                #except (NoSuchElementException,StaleElementReferenceException) as e:
                    #rating = -1 #You need to set a "not found value. It's important."
    
                #Printing for debugging
                if verbose:
                    print("Job Title: {}".format(job_title))
                    print("Salary Estimate: {}".format(salary_estimate))
                    print("Job Description: {}".format(job_description[:500]))
                    #print("Rating: {}".format(rating))
                    print("Company Name: {}".format(company_name))
                    print("Location: {}".format(location))
                    print("Job Function: {}".format(job_function))
    
                #Going to the Company tab...
                #clicking on this:
                #<div class="tab" data-tab-type="overview"><span>Company</span></div>
                #time.sleep(1)
                
                #try:
                 #   try:
                  #      driver.find_element_by_xpath('.//div[@id="SerpFixedHeader"]//span[text()="Company"]').click()
                  #  except (NoSuchElementException,StaleElementReferenceException) as e:
                   #     pass
    
                    #try:
                     #   size = driver.find_element_by_xpath('.//div[@id="EmpBasicInfo"]//span[text()="Size"]//following-sibling::*').text
                    #except (NoSuchElementException,StaleElementReferenceException) as e:
                     #   size = -1
    
                    #try:
                     #   founded = driver.find_element_by_xpath('.//div[@id="EmpBasicInfo"]//span[text()="Founded"]//following-sibling::*').text
                    #except (NoSuchElementException,StaleElementReferenceException) as e:
                     #   founded = -1
    
                    #try:
                     #   type_of_ownership = driver.find_element_by_xpath('.//div[@id="EmpBasicInfo"]//span[text()="Type"]//following-sibling::*').text
                    #except (NoSuchElementException,StaleElementReferenceException) as e:
                     #   type_of_ownership = -1
    
                    #try:                        
                     #   industry = driver.find_element_by_xpath('.//div[@id="EmpBasicInfo"]//span[text()="Industry"]//following-sibling::*').text
                    #except (NoSuchElementException,StaleElementReferenceException) as e:
                     #   industry = -1
    
                    #try:
                     #   sector = driver.find_element_by_xpath('.//div[@id="EmpBasicInfo"]//span[text()="Sector"]//following-sibling::*').text
                    #except (NoSuchElementException,StaleElementReferenceException) as e:
                     #   sector = -1
    
                    #try:                        
                     #   revenue = driver.find_element_by_xpath('.//div[@id="EmpBasicInfo"]//span[text()="Revenue"]//following-sibling::*').text
                    #except (NoSuchElementException,StaleElementReferenceException) as e:
                     #   revenue = -1
                        
    
                #except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                    #size = -1
                    #founded = -1
                    #type_of_ownership = -1
                    #industry = -1
                    #sector = -1
                    #revenue = -1
    
                    
                #if verbose:
                    
                    #print("Size: {}".format(size))
                    #print("Founded: {}".format(founded))
                    #print("Type of Ownership: {}".format(type_of_ownership))
                    #print("Industry: {}".format(industry))
                    #print("Sector: {}".format(sector))
                    #print("Revenue: {}".format(revenue))
                    # print("Headquarters: {}".format(headquarters))
                    # print("Competitors: {}".format(competitors))
                    #print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
                    
                    
                #if key == True:
                num = num+1
                jobs.append({"Job Title" : job_title,
                "Searched Job Title" : keyword,
                "Company Name" : company_name,
                "Full Description" : full_description,
                "Location" : location,
                #"Salary Estimate" : salary_estimate,
                "Job Function" : job_function,
                #"Job Description" : job_description,
                #"Rating" : rating,            
                #"Size" : size,
                #"Founded" : founded,
                #"Type of ownership" : type_of_ownership,
                #"Industry" : industry,
                #"Sector" : sector,
                #"Revenue" : revenue,
                "URL" : link})
                    
                print("----- {0} {1} ----- Page {2}".format(num,keyword,pg))
    
                currentJoblist=currentJoblist+1 # increasing the count of the list of buttons clicked and saved
                
                if not (currentJoblist < listButtonsCount): # to check the list last button and to go to next page
                        pg=pg+1
                        currentJoblist = 0  # resetting the button list count for new page button's list
                        break
                        
    
            #Clicking on the "next page" button
            try:                
                # driver.find_element_by_xpath('.//li[@class="next"]//a').click()
                
                driver.find_element_by_xpath('//*[@id="FooterPageNav"]//a[@data-test="pagination-next"]').click()
                
            except (NoSuchElementException,StaleElementReferenceException) as e:
                print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
                break

    driver.quit()
    
    executionTime = (time.time() - startTime)
    print('Execution time in seconds: ' + str(executionTime))
    pd.DataFrame(jobs).to_csv("/content/drive/MyDrive/data/glassdoor/{}.csv".format(job))
    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [14]:
df = pd.DataFrame()
no_of_jobs = 50
verb = False
sleept = 2

#get_jobs("Financial Planner", 5,verb,path,sleept)

# Extracting

In [9]:
titles = titles.dropna(axis=0, how='all', subset=['Job Title'])

In [10]:
titles = titles = titles.iloc[380:]

titles.shape

(22, 26)

In [15]:
import math

titles['NJ'] = ''
for rowIndex, row in titles.iterrows(): #iterate over rows
    k=0
    for columnIndex, value in row.items():
        value = str(value)
        if not (value=='NaN' or value == 'nan'):
            k=k+1
    titles['NJ'][rowIndex] = math.floor(20/k)

#df = pd.DataFrame()
for rowIndex, row in titles.iterrows(): #iterate over rows
    k=0
    for columnIndex, value in row.items():
        value = str(value)
        if not (value=='NaN' or value == 'nan'):
            #print(titles['NJ'][rowIndex])
            df1 = get_jobs(value, titles['NJ'][rowIndex],verb,path,sleept) 
        df = df.append(df1) 

WebDriverException: ignored

# Clean

In [ ]:
import wordninja

import re
import unicodedata
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.util import ngrams


# Function to generate n-grams from sentences.
def extract_ngrams(data):
    l = []
    for i in range(1,6):
        n_grams = ngrams(nltk.word_tokenize(data), i)
        l.append( [' '.join(grams) for grams in n_grams])
    flat_ls = list(itertools.chain(*l))
    return (flat_ls)


def remove_accented_chars(x):
    """The function changes the accented characters into their equivalent normal form,
    to do so, normalize function with 'NFKD' is used which replaces the compatibility characters into
    theri euivalent
    
  param x(str): the sentence in which accented characters are to be detected and removes
  return x(str): sentence with accented characters replaced by their equivalent"""
    
    x = unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return x


def preprocess(df,d):
    """Preprocesses the given document by applying the following functionalities
  lower: lowers all the characters for uniformity
  remove special characters: using regex, removes all the punctuations etc
  remove space: removes trailing spaces and extra spaces between words
  remove accented characters: change accented characters to its normal equivalent
  remove stop words"""
  
    df[d] = df[d].apply(lambda x: x.lower())
    df[d] = df[d].apply(lambda x: re.sub('[^A-Z a-z 0-9-]+', '', x))
    df[d] = df[d].apply(lambda x: " ".join(x.split()))
    df[d] = df[d].apply(lambda x: remove_accented_chars(x))
    df[d] = df[d].apply(lambda x: " ".join([t for t in x.split() if t not in STOP_WORDS]))


def salary_extract(x):
    #change to string
    if x == -1:
        return 'Missing Value'
    return x.astype(str)    
    
def get_experience(x):
    """
    param
     -x : Textfield from where experience is to be extracted
    return
     -item : Returns year of experience if available, or returns -1 if not available 
    """
    
    exp = re.findall('\\d+\\s+(?:months?|years?)',x)
    if exp:
        for item in exp:
            return item
    else:
        return ('-1')    

def get_language(x):

    """
    Matched field with with language dictionary and returns languages if present in feild
    
    param
     -x : Textfield from where language is to be extracted
    return
     -item : Returns language if available, or returns -1 if not available 
    """
    
    language_dictionary = ['english',
                           'french',
                           'spanish',
                           'chinese',
                           'german',
                           'mandarin',
                           'japanese',
                           'russian']
    lang = ''
    
    for word in language_dictionary:
        if word in x:
            lang = lang + "|" + word
    if lang == '':
        return ('-1')
    else:
        return lang


def split_(k):
    l = wordninja.split(k)
    x = ''
    for word in l:
        x = x+word+' '
    return (x)

def get_responsibility(x):
    res = ['responsibilities',
           'responsibility',
           'function',
           'activities',
           'activity',
           'responsible',
           'role',
           'roles',
           'day',
           'workday',
           'day to day']
    flag = False
    #x = x.split()
    #print(x)
    
    for word in res:
        #print(word)
        try:
            pos = x.index(word)
            flag = True
            return (x[pos+len(word):pos+300])
        except ValueError:
            flag = False
    return ('No Value')

def get_skills(x):
    res = ['sucessful candidates',
           'skills',
           'expectation',
           'ideal candidate will have',
           'ideal candidate will possess',
           'skills you have',
           'you possess',
           'skill',
           'requirement',
           'you are an ideal candidate if',
           'skills required']
    flag = False
    #x = x.split()
    #print(x)
    
    for word in res:
        #print(word)
        try:
            pos = x.index(word)
            flag = True
            return (x[pos+len(word):pos+300])
        except ValueError:
            flag = False
    return ('No Value')


import re

certis = {}
for i in df['Job Title']:
    certis[i] = ""

cert = pd.read_csv('C:\\Users\\Dell\\OneDrive\\DataR Labs\\Resume_Parser\\certificates.csv') 
for i in range(0,len(cert)):
    certis[cert['Job'][i].rstrip().lstrip()] = cert['Certificates'][i]


certifications = []
for ind in range(0,len(df)):
    
    for cer in certis[df['Job Title'][ind]].split(","):
        certitext = ''
        
        if(re.search(cer.lstrip().rstrip(),df['Full Description'][ind])):
            certitext = cer+ " "+certitext 
        
    if(certitext==''):
            certitext= 'No Value'
        
    certifications.append(certitext)
    
    
educ = []
educate = pd.read_csv('EducationWords.csv')
#educate = pd.read_csv('/content/drive/MyDrive/data/EducationWords.csv') 
for i in educate['Education']:
    educ.append(i)

def get_education(x):
    educa =""
    education = []
    for ed in educ:
        if(re.search(ed.lower(),x.lower())):
            educa = ed+" "+educa
            education.append(educa)
            
    if len(education)!=0:
        return (education[-1])
    else:
        return 'No Value'

In [ ]:
df['Full Description'] = df['Full Description'].astype(pd.StringDtype())
preprocess(df,'Full Description')
df['Full Description'] = df['Full Description'].apply(split_)
df['Experience in years'] = df['Full Description'].apply(get_experience)
df['Experience in years'] = df['Experience in years'].map(lambda x : x.split(' ')[0])
df['bow'] = df['Full Description'].apply(extract_ngrams)
df['Foreign Language'] = df['Full Description'].apply(get_language)
df['Responsibilities'] = df['Full Description'].apply(get_responsibility)
df['Skills'] = df['Full Description'].apply(get_skills)
df['Certification'] = certifications
df = df.replace(' ','No Value')
df['Education'] = df['Full Description'].apply(get_education)